# The Questions

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

bool_df = pd.read_csv("recipeBool.csv")
bool_df.index = bool_df["Unnamed: 0"]
bool_df = bool_df.drop(["Unnamed: 0"], axis = 1)
columns = bool_df.columns

## Bei Einrichtung
1. Frage: Willst du einen Thermomix verwenden?
2. Frage: Nenne Zutaten, welche du nicht verwenden möchtest.
3. Frage: Hast du irgendwelche Alergien?
4. Frage: Besondere Ernährungsweise?
5. Frage: Wie bewertest du folgende 5 Gerichte? (0-5)


Funktion zu Erstellung eines Userprofiles:

In [3]:
#creating helping df´s for finden id´s

con = sqlite3.connect("RecipeDB.db")
# ing_df = pd.read_sql("Select * FROM IngFamily", con)
# temp = pd.read_sql('SELECT * FROM Ingredients',con)
# ing_df.index = ing_df["familyID"]
# for x in temp.itertuples():
#     ing_df.loc[x.familyID]['familyName']+= ' '+x.ingName
# ing_df = ing_df.drop(["familyID"], axis = 1)

# ing_df.rename(columns = {'familyName':'info'}, inplace = True)

ing_df = pd.read_sql("Select ingID, ingName, familyName FROM Ingredients NATURAL JOIN IngFamily", con)
ing_df.index = ing_df['ingID']
ing_df['info'] = ing_df['ingName'] +" "+ ing_df['familyName'];
ing_df = ing_df.drop(['ingName'], axis = 1)
ing_df = ing_df.drop(['familyName'], axis = 1)
ing_df = ing_df.drop(['ingID'], axis = 1)


tags_df = pd.read_sql("Select * from Tags", con)
tags_df.index = tags_df["tagID"]
tags_df['info'] = tags_df['tagName']+' '+tags_df['tagType']
tags_df = tags_df.drop(["tagID"], axis = 1)
tags_df = tags_df.drop(["tagName"], axis = 1)
tags_df = tags_df.drop(["tagType"], axis = 1)

alg_df = pd.read_sql("Select * from Allergens", con)
alg_df.index = alg_df["alleID"]
alg_df['info']= alg_df['alleType']+' '+alg_df['alleName']+' '+alg_df['alleSlug']
alg_df = alg_df.drop(["alleID"], axis = 1)
alg_df = alg_df.drop(["alleType"], axis = 1)
alg_df = alg_df.drop(["alleName"], axis = 1)
alg_df = alg_df.drop(["alleSlug"], axis = 1)

thermo = tags_df[tags_df["info"].str.contains("thermomix")]




In [4]:
alg_df.head()

,info
alleID,
57962a07b7e8697d4b3052f5,gluten Weizen weizen
57962a07b7e8697d4b3052f3,maycontaintracesofallergens kann Spuren von Al...
57962a07b7e8697d4b3052fa,milk Milch laktose
57962a07b7e8697d4b3052fd,mustard Senf senf
57962a07b7e8697d4b3052f6,egg Ei ei


In [5]:
ing_df

,info
ingID,
554b7698fd2cb96c628b4568,Arborio-Reis Reis
55a7a8f8fd2cb9570f8b4567,Arabisches Fladenbrot Brot
556c4ed44dab710b3d8b4569,Apfelessig Essig
55535c2efd2cb9ae048b4567,Aubergine Aubergine
556db2a04dab71104b8b4568,Naan-Brot Brot
...,...
5bf2c4e9ae08b5103d1caae2,Schinkenbraten Schweinefleisch
5c1227bee3f33930ed6f7d42,Apfelsaft Fruchtsaft
5c0e62fac445fa5e3d20aa72,Hühnerbrühe Stock


In [6]:
tags_df.head()

,info
tagID,
57d13370b7e8697d4b30531a,leichter Genuss low-calorie
57d13370b7e8697d4b305319,scharf spicy
58aeb96c99052d5fee3dd712,Chef's Choice chefs-choice
57d13370b7e8697d4b305317,Vegetarisch vegetarian
57d13370b7e8697d4b30530e,Zeit sparen under-30-minutes


In [7]:
test = (False, ["Schwein", "Tomaten", "Schimmelkäse"], ["Laktose", "Schwefel"], ["Proteine", "fettarm"])
def create_Userprofile(tmix, ing, alg, tags):
    data = np.full([1, len(columns)], 0.5, dtype=np.float64)
    user_Profile = pd.DataFrame(data=data, columns = columns)
    for x in thermo.itertuples():
        user_Profile[x.Index] = 1.0 if tmix else 0.0
    for x in ing:
        for i in ing_df[ing_df['info'].str.contains(x)].itertuples():
            #TODO sachen wie gewürzmischungen oder komplimentär zutaten werden auch gefunden 
            print('ing :',x,' id info', i.Index, ' ', i.info)
            user_Profile[i.Index] = 0;
            
    for a in alg:
        for i in alg_df[alg_df['info'].str.contains(a)].itertuples():
            print('allergie :',a,' id info', i.Index, ' ', i.info)
            user_Profile[i.Index] = 0.0
            
    for tag in tags:
        for i in tags_df[tags_df['info'].str.contains(tag)].itertuples():
            print('tag :',tag,' id info', i.Index, ' ', i.info)
            user_Profile[i.Index] = 1.0
    return user_Profile.iloc[0]

user_p = create_Userprofile(test[0], test[1], test[2], test[3])
    
    
    
    
    

ing : Schwein  id info 554b6b664dab71160d8b4569   Schweinefilet Schweinefleisch
ing : Schwein  id info 555dabaff8b25e246a8b4568   Speckwürfel Schweinefleisch
ing : Schwein  id info 556842e3f8b25efe3e8b456a   Schinkenspeck Schweinefleisch
ing : Schwein  id info 55535376f8b25e8a038b4568   Bacon Schweinefleisch
ing : Schwein  id info 556881724dab71111f8b4568   Schweineschnitzel Schweinefleisch
ing : Schwein  id info 556c1b03f8b25eac648b4567   Chorizo Schweinefleisch
ing : Schwein  id info 556c771bfd2cb9604e8b4568   Schweinenacken Schweinefleisch
ing : Schwein  id info 5566efd9fd2cb952018b456a   Schweinefleisch Schweinefleisch
ing : Schwein  id info 555341defd2cb92b1a8b456a   Schweinerückensteak Schweinefleisch
ing : Schwein  id info 55704ae24dab71b7688b4569   Schweinelachssteaks Schweinefleisch
ing : Schwein  id info 557050784dab71b7688b456a   Pancetta Schweinefleisch
ing : Schwein  id info 5582c296f8b25e02648b4567   Pulled Pork Schweinefleisch
ing : Schwein  id info 55e84be279a23c827e8b4

## Bei Normaler Verwendung
### 1. Option:
Empfehlung eines Rezeptes

### 2. Option:
Spezifischere Empfehlung durch folgende Frage:
1. Nenne Zutaten, welche du gerne verwenden möchtest.

In [8]:
bool_df.shape

(1897, 871)

## TODO Knowledge Base Recommender

In [9]:
def know_recommender(user_profile):
    res = bool_df.dot(user_profile)
    for i in res.nsmallest(1897).iteritems():
        yield i
        

In [10]:
#print(user_p)
count  = 0;
for i in know_recommender(user_p):
    count += 1
    print(i)
    if count == 5:
        break

('5c30c44ec445fa3dbb5a80b2', 0.0)
('5b22766930006c7d844dce42', 0.0)
('5b22766dae08b516df4039d2', 0.0)
('5b2a53b4ae08b547b734ca82', 0.0)
('5b2a6196ae08b552961e98a2', 0.0)


## TODO Contentend based Recommender


## TODO Hybrid Recommender


## TODO Recipe to JSON parser

In [14]:
import json 
con = sqlite3.connect("RecipeDB.db")
cur = con.cursor();

def recipe_json(recipe_id):
    res = {}
    res['id'] = recipe_id
    sql = """ SELECT * FROM Recipe WHERE recID = ? ;"""
    recs = cur.execute(sql,(recipe_id,)).fetchall()[0]
    #basic infos
    res['name'] = recs[1]
    res['headline'] = recs[2]
    res['disc'] = recs[3]
    #ingredients
    recs = cur.execute("SELECT ingID, ingName, familyID, familyName, amount, unit FROM IngInRec NATURAL JOIN Ingredients NATURAL JOIN IngFamily WHERE recID = ? ;",(recipe_id,)).fetchall()
    ingr = {}
    for ing in recs:
        ingr[ing[0]] = {'name':ing[1],'amount': ing[4], 'unit': ing[5], 'family' : {'id':ing[2], 'name': ing[3]}}
    res['ingredients'] = ingr
    
    #nutr
    recs = cur.execute("SELECT nType, amount, unit, nName FROM NutrInRec NATURAL JOIN Nutration WHERE recID = ? ;",(recipe_id,)).fetchall()
    nutr = {}
    for i in recs:
        nutr[i[0]] = {'amount' : i[1], 'unit' : i[2], 'name' : i[3]}
    res['nutration'] = nutr
    
    #allergene
    recs = cur.execute("SELECT alleID, alleName FROM AlleInRec NATURAL JOIN Allergens WHERE recID = ? ;",(recipe_id,)).fetchall()
    alle = {}
    for i in recs:
        alle[i[0]] = i[1]
    res['allergies'] = alle
    
    #tags
    
    recs = cur.execute("SELECT tagID, tagName FROM TagsInRec NATURAL JOIN Tags WHERE recID = ? ;",(recipe_id,)).fetchall()
    tags = {}
    for t in recs:
        tags[t[0]] = t[1]
    res['tags'] = tags
    
    #steps
    recs = cur.execute("SELECT stepIndex, stepDisc from StepsInRec WHERE recID = ?  ORDER BY stepIndex ;", (recipe_id,)).fetchall();
    
    res['steps'] = [i[1]for i in recs]
    
    return json.dumps(res, indent=2, ensure_ascii=False)
    
count = 0
count2 = 0
for i in know_recommender(user_p):
    if i[1]>0.0:
        count2+=1
        print(recipe_json(i[0]))
        break
        
    else:
        count += 1
        #print(recipe_json(i[0]))
    
print(count," ",count2, " both ", (count+count2))
# print(recipe_json("53f45a29ff604dac328b456e"))

{
  "id": "5538c9d46ced6eeb208b4570",
  "name": "Mit Tomate und Basilikum gefüllte Manti",
  "headline": "in erfrischender Sumach-Joghurt-Soße",
  "disc": "Döner kennt jeder, dabei hat die türkische Küche noch viel mehr zu bieten, zum Beispiel Manti, türkische Ravioli. In dieser vegetarischen Variante sind sie mit Tomate und Basilikum gefüllt und werden mit dem in der Türkei sehr beliebten Gewürz Sumach serviert. Afiyet olsun!",
  "ingredients": {
    "554b268ffd2cb983608b4568": {
      "name": "Knoblauch",
      "amount": 0.0,
      "unit": "Einheit",
      "family": {
        "id": "58877cf3ed71a76e9c03fba4",
        "name": "Knoblauch"
      }
    },
    "554b34b64dab71aa058b4573": {
      "name": "Petersilie",
      "amount": 2.5,
      "unit": "g",
      "family": {
        "id": "58877cf3ed71a76e9c03fbf9",
        "name": "Petersilie"
      }
    },
    "554b7fb3fd2cb9d8608b4571": {
      "name": "Sahne-Joghurt",
      "amount": 100.0,
      "unit": "g",
      "family": {
       